In [23]:
from tables import *
h5file = open_file("data/satellite data/export_test.h5", mode="r", title="TLE Indexable Data")


In [24]:
h5file.root.readout

/readout (Table(124317972,)) 'Main Listing'
  description := {
  "epoch": Float64Col(shape=(), dflt=0.0, pos=0),
  "norad_id": Int64Col(shape=(), dflt=0, pos=1),
  "line_one": StringCol(itemsize=80, shape=(), dflt=b'', pos=2),
  "line_two": StringCol(itemsize=80, shape=(), dflt=b'', pos=3)}
  byteorder := 'little'
  chunkshape := (372,)

In [25]:
import random
import time

def load_row():
    nrows = h5file.root.readout.shape[0]
    for i in range(100):
        idx = random.randint(0, nrows)
        h5file.root.readout[idx]

In [26]:
timeit load_row()

74.3 ms ± 1.7 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [27]:
# Load time is _very_ fast. (~1ms per record)

In [28]:
from skyfield.sgp4lib import EarthSatellite
nrows = h5file.root.readout.shape[0]
def get_random_tle():
    idx = random.randint(0, nrows)
    date, sat, tle1, tle2 = h5file.root.readout[idx]
    return EarthSatellite(tle1.decode(), tle2.decode())

In [29]:
timeit get_random_tle()

821 µs ± 19.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [30]:
idx = random.randint(0, nrows)
date, sat, tle1, tle2 = h5file.root.readout[-1]

In [31]:
import numpy as np
import datetime
from skyfield import api
ts = api.load.timescale()

def one_day_of_tle_projection():
    sat = get_random_tle()
    jds = ts.tt_jd(np.linspace(sat.epoch.tt - 0.5, sat.epoch.tt +0.5, 24*60))
    sat.at(jds)

In [32]:
timeit one_day_of_tle_projection()

226 ms ± 1.68 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [33]:
# Thats a bit better. As Peter suggested, pulling out the DT math and seeing how that goes...

In [34]:
from skyfield.framelib import ecliptic_J2000_frame
sat = get_random_tle()
jds = ts.tt_jd(np.linspace(sat.epoch.tt - 0.5, sat.epoch.tt +0.5, 24*60))

def do_the_math():
    return sat.at(jds).frame_latlon(ecliptic_J2000_frame)

In [35]:
timeit do_the_math()

3.69 ms ± 46.1 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [36]:
# _much_ faster!